Please use the attached data file to construct a K-nearest neighbor (KNN) classifier with various different numbers (remember k has to be odd), starting from k = 1 to some numbers (preferably a large range).

**Do the following:**

1. Use the 5-fold Cross-Validation (CV) method to find out the Expected Prediction Error (EPE) using KNN for different k, please also compute the Residual Sum of Squares (RSS).
2. Put these EPE and RSS errors vs. k (it has to be at least more than 10 different values of k) on a plot.
3. According to your plot (which is a bias-variance trade-off plot for model complexity), answer the following questions:
   1. How does the value of k control the model complexity?
   2. What's the optimal k?

*The code will have to be in a Jupyter notebook file when you submit.*

In [2]:
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.stats import t
import pandas as pd
import statsmodels.api as sm
import itertools

ModuleNotFoundError: No module named 'sklearn'